In [1]:
import numpy as np 
import pandas as pd 
import keras 
import jieba
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import scipy.sparse as sp
import gensim
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Activation,Dropout
from keras.layers import Dense,Dropout,Activation,Input,Convolution1D,Conv1D,GlobalAveragePooling1D,GlobalMaxPooling1D,MaxPooling1D,Flatten,concatenate,Embedding,GRU,Lambda, LSTM, TimeDistributed
from keras.layers.merge import Concatenate
from keras.models import model_from_json
from keras.preprocessing.text import Tokenizer
from keras.utils.np_utils import to_categorical
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import RMSprop
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.models import load_model
from keras.layers import Dense, Embedding, LSTM,Flatten
from keras.models import Model
from gensim.models import word2vec
import time
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 1000)
np.set_printoptions(threshold=100)

Using TensorFlow backend.
E:\Programfile\Anaconda3\envs\tensorflow\lib\site-packages\gensim\utils.py:865: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
age_train = pd.read_csv('age_train.csv',names=['uid','labels'])
age_test = pd.read_csv('age_test.csv',names=['uid'])
#app_info = pd.read_csv('app_info.csv', names=['appId','category'])
user_app_actived = pd. read_csv('user_app_actived.csv',names=['uid','appId'])
user_app_actived['appId_list'] = user_app_actived['appId'].apply(lambda x : x.split('#'))
age_train = age_train[:1000000]
train_data = age_train.merge(user_app_actived,on='uid')
test_data = age_test.merge(user_app_actived,on='uid')

In [ ]:
embed_size = 128
fastmodel = word2vec.Word2Vec(list(user_app_actived['appId_list']), size=embed_size, window=4, min_count=5, negative=2,
                     sg=1, sample=0.002, hs=1, workers=4)

fastmodel.save('./word2vec.m')
print('finished and saved!')

embedding_fast = pd.DataFrame([fastmodel[word]
                               for word in (fastmodel.wv.vocab)])
embedding_fast['app'] = list(fastmodel.wv.vocab)
embedding_fast.columns = ["fdim_%s" %
                          str(i) for i in range(embed_size)]+["app"]

In [4]:
fastmodel = gensim.models.Word2Vec.load('./word2vec.m')

In [3]:
tokenizer = Tokenizer(lower=False, char_level=False, split='#')
tokenizer.fit_on_texts(list(user_app_actived['appId']))
X_seq = tokenizer.texts_to_sequences(train_data['appId'])
X_test_seq = tokenizer.texts_to_sequences(test_data['appId'])
train_y = train_data['labels']
del train_data
del test_data
del user_app_actived
maxlen = 879
X = pad_sequences(X_seq,maxlen= maxlen ,value=0)
X_test = pad_sequences(X_test_seq,maxlen=maxlen ,value=0)


In [5]:
embed_size = 128
max_feaures = 9401
embedding_matrix = np.zeros((max_feaures, embed_size))
for word in tokenizer.word_index:
    if word not in fastmodel.wv.vocab:
        continue
    embedding_matrix[tokenizer.word_index[word]] = fastmodel[word]



In [ ]:
def model_age_conv(embedding_matrix):

    # The embedding layer containing the word vectors
    K.clear_session()
    emb_layer = Embedding(
        input_dim=embedding_matrix.shape[0],
        output_dim=embedding_matrix.shape[1],
        weights=[embedding_matrix],
        input_length=maxlen,
        trainable=False
    )
    lstm_layer = Bidirectional(
        GRU(128, recurrent_dropout=0.15, dropout=0.15, return_sequences=True))

    # 1D convolutions that can iterate over the word vectors
    conv1 = Conv1D(filters=128, kernel_size=1,
                   padding='same', activation='relu',)
    conv2 = Conv1D(filters=64, kernel_size=2,
                   padding='same', activation='relu', )
    conv3 = Conv1D(filters=64, kernel_size=3,
                   padding='same', activation='relu',)
    conv5 = Conv1D(filters=32, kernel_size=5,
                   padding='same', activation='relu',)

    # Define inputs
    seq = Input(shape=(maxlen,))

    # Run inputs through embedding
    emb = emb_layer(seq)

    lstm = lstm_layer(emb)
    # Run through CONV + GAP layers
    conv1a = conv1(lstm)
    gap1a = GlobalAveragePooling1D()(conv1a)
    gmp1a = GlobalMaxPool1D()(conv1a)

    conv2a = conv2(lstm)
    gap2a = GlobalAveragePooling1D()(conv2a)
    gmp2a = GlobalMaxPool1D()(conv2a)

    conv3a = conv3(lstm)
    gap3a = GlobalAveragePooling1D()(conv3a)
    gmp3a = GlobalMaxPooling1D()(conv3a)

    conv5a = conv5(lstm)
    gap5a = GlobalAveragePooling1D()(conv5a)
    gmp5a = GlobalMaxPooling1D()(conv5a)

    merge1 = concatenate([gap1a, gap2a, gap3a, gap5a])

    # The MLP that determines the outcome
    x = Dropout(0.3)(merge1)
    x = BatchNormalization()(x)
    x = Dense(200, activation='relu',)(x)
    x = Dropout(0.22)(x)
    x = BatchNormalization()(x)
    x = Dense(200, activation='relu',)(x)
    x = Dropout(0.22)(x)
    x = BatchNormalization()(x)
    x = Dense(200, activation='relu',)(x)
    x = Dropout(0.22)(x)
    x = BatchNormalization()(x)
    pred = Dense(11, activation='softmax')(x)

    model = Model(inputs=seq, outputs=pred)
    model.compile(loss='categorical_crossentropy',
                  optimizer=AdamW(weight_decay=0.08,))

    return model


In [18]:
model = Sequential()

seq = Input(shape=[maxlen])
# 使用Embedding层将每个词编码转换为词向量
#Embedding layers
emb = Embedding(len(embedding_matrix),       #表示文本数据中词汇的取值可能数,从语料库之中保留多少个单词。 因为Keras需要预留一个全零层， 所以+1
                            embed_size,       # 嵌入单词的向量空间的大小。它为每个单词定义了这个层的输出向量的大小
                            weights=[embedding_matrix], #构建一个[num_words, EMBEDDING_DIM]的矩阵,然后遍历word_index，将word在W2V模型之中对应vector复制过来。换个方式说：embedding_matrix 是原始W2V的子集，排列顺序按照Tokenizer在fit之后的词顺序。作为权重喂给Embedding Layer
                            input_length=maxlen,     # 输入序列的长度，也就是一次输入带有的词汇个数
                            trainable=True        # 我们设置 trainable = False，代表词向量不作为参数进行更新
                    )(seq)

convs = []
filter_sizes = [2,]
filter_nums = [128,]
for fsz,fnums in zip(filter_sizes,filter_nums):
    conv1 = Conv1D(filters=fnums,kernel_size=fsz,activation='relu')(emb)
    pool1 = MaxPooling1D(int(conv1.shape[1]))(conv1)
    pool1 = Flatten()(pool1)
    convs.append(pool1)
#merge = concatenate(convs,axis=1)
#out = Dropout(0.2)(merge)
out = Dropout(0.2)(convs[0])

output = Dense(64,activation='relu')(out)

output = Dense(units=6,activation='sigmoid')(output)

model = Model([seq],output)
model.compile(loss='categorical_crossentropy',optimizer='RMSprop',metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, 879)               0         
_________________________________________________________________
embedding_8 (Embedding)      (None, 879, 128)          1203328   
_________________________________________________________________
conv1d_11 (Conv1D)           (None, 878, 128)          32896     
_________________________________________________________________
max_pooling1d_8 (MaxPooling1 (None, 1, 128)            0         
_________________________________________________________________
flatten_9 (Flatten)          (None, 128)               0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 64)                8256      
__________

In [19]:
train_y_cat = to_categorical(train_y-1,num_classes=6)

In [ ]:
earlyStopping = keras.callbacks.EarlyStopping(
			monitor='val_loss',
			patience=0,  # 当early stop被激活（如发现loss相比上一个epoch训练没有下降），则经过patience个epoch后停止训练。
			verbose=0,
			mode='auto')

print("training model")
model.fit(X,train_y_cat,validation_split=0.02,batch_size=32,epochs=10,verbose=1,callbacks=[earlyStopping],shuffle=True,class_weight='auto') #自动设置class weight让每类的sample对损失的贡献相等


training model
Train on 980000 samples, validate on 20000 samples
Epoch 1/10
918016/980000 [===========================>..] - ETA: 1:09 - loss: 1.5275 - acc: 0.2192

In [ ]:
X[0][-19]

In [ ]:
train_data.reset_index(inplace = True)

In [ ]:
train_data.loc[0]['appId_list'][-19]

In [ ]:
X[169568][850]

In [ ]:
tokenizer.word_index['a00184278']

In [ ]:
train_y.value_counts().plot.bar()

In [ ]:
def cnn_w2v(y_cat_nums,max_features,embedding_dims,filters,maxlen):
	# CNN参数
	kernel_size = 3

	w2v_model = gensim.models.Word2Vec.load('./word2vec.m')
	word2idx = {"_PAD": 0}  # 初始化 `[word : token]` 字典，后期 tokenize 语料库就是用该词典。
	vocab_list = [(k, w2v_model.wv[k]) for k, v in w2v_model.wv.vocab.items()]
	# 存储所有 word2vec 中所有向量的数组，留意其中多一位，词向量全为 0， 用于 padding
	embeddings_matrix = np.zeros((len(w2v_model.wv.vocab.items()) + 1, w2v_model.vector_size))
	print('Found %s word vectors.' % len(w2v_model.wv.vocab.items()))
	for i in range(len(vocab_list)):
		word = vocab_list[i][0]
		word2idx[word] = i + 1
		embeddings_matrix[i + 1] = vocab_list[i][1]

	model = Sequential()
	# 使用Embedding层将每个词编码转换为词向量
	model.add(Embedding(len(embeddings_matrix),       #表示文本数据中词汇的取值可能数,从语料库之中保留多少个单词。 因为Keras需要预留一个全零层， 所以+1
								embedding_dims,       # 嵌入单词的向量空间的大小。它为每个单词定义了这个层的输出向量的大小
								weights=[embeddings_matrix], #构建一个[num_words, EMBEDDING_DIM]的矩阵,然后遍历word_index，将word在W2V模型之中对应vector复制过来。换个方式说：embedding_matrix 是原始W2V的子集，排列顺序按照Tokenizer在fit之后的词顺序。作为权重喂给Embedding Layer
								input_length=maxlen,     # 输入序列的长度，也就是一次输入带有的词汇个数
								trainable=True        # 我们设置 trainable = False，代表词向量不作为参数进行更新
						))
	model.add(Conv1D(filters,
					 kernel_size,
					 padding='valid',
					 activation='relu',
					 strides=1))
	# 池化
	model.add(GlobalMaxPooling1D())

	model.add(Dense(y_cat_nums, activation='softmax')) #第一个参数units: 全连接层输出的维度，即下一层神经元的个数。
	model.add(Dropout(0.2))
	model.compile(loss='categorical_crossentropy',
				  optimizer='adam',
				  metrics=['accuracy'])

	model.summary()

	return model,word2idx
cnn_model,word2idx = cnn_w2v(6,100,100,200,879)

In [ ]:
w2v_model = gensim.models.Word2Vec.load('./word2vec.m')

In [ ]:
vocab_list = [(k, w2v_model.wv[k]) for k, v in w2v_model.wv.vocab.items()]

In [ ]:
vocab_list[:3]

In [ ]:
tokenizer.word_index

In [ ]:
train_y_cat = to_categorical(total_data[:1000000]['labels']-1,num_classes=6)

In [ ]:
earlyStopping = keras.callbacks.EarlyStopping(
			monitor='val_loss',
			patience=1,  # 当early stop被激活（如发现loss相比上一个epoch训练没有下降），则经过patience个epoch后停止训练。
			verbose=0,
			mode='auto')

print("training model")
cnn_model.fit(X[:1000000],train_y_cat,validation_split=0.2,batch_size=32,epochs=10,verbose=1,callbacks=[earlyStopping],shuffle=True,class_weight='auto') #自动设置class weight让每类的sample对损失的贡献相等


In [ ]:
train_X.shape

In [9]:
def test_cnn(y,maxlen,max_features,embedding_dims,filters = 250):
	#Inputs
	seq = Input(shape=[maxlen],name='x_seq')

	#Embedding layers
	emb = Embedding(max_features,embedding_dims)(seq)

	# conv layers
	convs = []
	filter_sizes = [2,3,4,5]
	for fsz in filter_sizes:
		conv1 = Conv1D(filters,kernel_size=fsz,activation='tanh')(emb)
		pool1 = MaxPooling1D(maxlen-fsz+1)(conv1)
		pool1 = Flatten()(pool1)
		convs.append(pool1)
	merge = concatenate(convs,axis=1)

	out = Dropout(0.2)(merge)
	output = Dense(32,activation='relu')(out)

	output = Dense(units=y.shape[1],activation='sigmoid')(output)

	model = Model([seq],output)
	model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
	return model
